# Evaluation notebook 

In [ ]:

from pprint import pprint
import os
import json
import pandas as pandas

In [2]:
def read_file(file_path):
    with open(file_path,'r', encoding="utf8") as file:
        return file.readlines()

def read_file_as_text(file_path):
    with open(file_path,'r', encoding="utf8") as file:
        return file.read()

def read_reference_file_for_bleu_metric(path):
    with open(path,'r', encoding="utf8") as file:
        # Returns a list of lists (each sentence is a list itself, with one item)
        return [list(map(str, line.rstrip(",\r\n").split(","))) for line in file]

In [3]:
p1 = 'simplify_summary'
p2 = 'summary_simplify'

In [4]:
def calculate_metrics_for_dataset(metric, pipeline=p1, data_dir='data/'):
    '''
    General method that takes in the metric method and returns the metric scores
    '''
    evaluation_dataset = dict()
    metric_name = metric.__name__
    wiki_base_dir = os.path.join(data_dir, 'wiki-auto')

    processed_articles = read_file_as_text('data/processed_articles.txt').split('\n')

    # base data/wiki-auto
    for folder in processed_articles:
        base_data_dir = os.path.join(wiki_base_dir, folder)
        ground_truth_file_path = os.path.join(base_data_dir, 'destination.txt')
        pipeline_dir = os.path.join(base_data_dir, pipeline)
        simplified_summary_file_path = os.path.join(pipeline_dir, 'simplified_summary.txt')
        if os.path.exists(simplified_summary_file_path):
            evaluation_dataset[folder] = metric(simplified_summary_file_path, ground_truth_file_path)
            print(f'Done generating {metric_name} scores for {folder}\r', end='', flush=True)
    with open(f'{data_dir}{pipeline}_{metric_name}', 'w') as outputfile:
        json.dump(evaluation_dataset, outputfile)
    print('\nDone')
    return evaluation_dataset

## Lexical similarity based scores - ROUGE

### ROUGE - ROUGE-1, ROUGE-2, ROUGE-L and Google ROUGE

In [ ]:
!pip install rouge
!pip install pandas
!pip install rouge-score

In [5]:
from rouge_score import rouge_scorer
from rouge import Rouge

In [6]:
def rouge_scores(hyp_file_path, ref_file_path):
    '''
    Calculate ROUGE-1, ROUGE-2 and ROUGE-l scores for hypothesis and reference files
    '''
    rouge = Rouge()
    hypotheses = ''
    references = ''
    hypotheses = read_file_as_text(hyp_file_path)
    references = read_file_as_text(ref_file_path)
    return rouge.get_scores(hypotheses, references, avg=True)

# pprint(calculate_rouge_score('data/wiki-auto/25/simplify_summary/simplified_summary.txt','data/wiki-auto/25/destination.txt'))
# calculate_rouge_score('data/wiki-auto/25/summary_simplify/simplified_summary.txt','data/wiki-auto/25/destination.txt')


def convert_google_rouge_format_to_usable_format(scores):
    google_rouge_stats = dict(
    {
        'rouge-1': {
            "r": 0,
            "p": 0,
            "f": 0,
        },
        'rouge-2': {
            "r": 0,
            "p": 0,
            "f": 0,
        },
        'rouge-l': {
            "r": 0,
            "p": 0,
            "f": 0,
        },
        'rouge-lsum': {
            "r": 0,
            "p": 0,
            "f": 0,
        }
        
    })
    for key in scores:
        if key == 'rouge1':
            google_rouge_stats['rouge-1'] = {"p":scores[key].precision,"r":scores[key].recall,"f":scores[key].fmeasure}
        if key == 'rouge2':
            google_rouge_stats['rouge-2'] = {"p":scores[key].precision,"r":scores[key].recall,"f":scores[key].fmeasure}
        if key == 'rougeL':
            google_rouge_stats['rouge-l'] = {"p":scores[key].precision,"r":scores[key].recall,"f":scores[key].fmeasure}
        if key == 'rougeLsum':
            google_rouge_stats['rouge-lsum'] = {"p":scores[key].precision,"r":scores[key].recall,"f":scores[key].fmeasure}
    return google_rouge_stats

def google_rouge_scores(hyp_file_path, ref_file_path):
    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL','rougeLsum'], use_stemmer=True)
    hypotheses = read_file_as_text(hyp_file_path)
    references = read_file_as_text(ref_file_path)
    scores = scorer.score(references,hypotheses)
    # pprint(type(scores))
    # return scores
    return convert_google_rouge_format_to_usable_format(scores)

In [7]:
# Test 
pprint(google_rouge_scores('data/wiki-auto/25/simplify_summary/simplified_summary.txt','data/wiki-auto/25/destination.txt'))
# pprint(calculate_google_rouge_score('data/wiki-auto/25/summary_simplify/simplified_summary.txt','data/wiki-auto/25/destination.txt'))

{'rouge-1': {'f': 0.5666280417149478,
             'p': 0.6680327868852459,
             'r': 0.4919517102615694},
 'rouge-2': {'f': 0.1654592871631411,
             'p': 0.1950802869832593,
             'r': 0.14364779874213837},
 'rouge-l': {'f': 0.14629200463499423,
             'p': 0.17247267759562843,
             'r': 0.12701207243460766},
 'rouge-lsum': {'f': 0.5559096176129779,
                'p': 0.655396174863388,
                'r': 0.48264587525150904}}


### Normal ROUGE

In [9]:
%%time
# Calculate ROUGE score for the whole dataset for the first pipeline (Simplify & Summarize)
calculate_metrics_for_dataset(rouge_scores, pipeline=p1)

CPU times: user 10min 5s, sys: 540 ms, total: 10min 6s
Wall time: 10min 6s


In [10]:
%%time
# Calculate ROUGE score for the whole dataset for the second pipeline (Summarize & Simplify)
calculate_metrics_for_dataset(rouge_scores, pipeline=p2)

CPU times: user 9min 55s, sys: 724 ms, total: 9min 56s
Wall time: 9min 57s


#### Preprocess the ROUGE scores files to calculate the average score for the whole dataset

In [11]:
%%time
simplify_summary_df = pandas.read_json(
    'data/simplify_summary_rouge_scores',
    encoding='utf8',
    orient='index'
)
display(simplify_summary_df)

summary_simplify_df = pandas.read_json(
    'data/summary_simplify_rouge_scores',
    encoding='utf8',
    orient='index'
)
display(summary_simplify_df)

,rouge-1,rouge-2,rouge-l
1004,"{'r': 0.202247191011235, 'p': 0.19148936170212...","{'r': 0.051204819277108, 'p': 0.05151515151515...","{'r': 0.18539325842696602, 'p': 0.175531914893..."
1005,"{'r': 0.234042553191489, 'p': 0.27049180327868...","{'r': 0.081481481481481, 'p': 0.11398963730569...","{'r': 0.20567375886524802, 'p': 0.237704918032..."
10065,"{'r': 0.23333333333333303, 'p': 0.233333333333...","{'r': 0.07042253521126701, 'p': 0.070422535211...","{'r': 0.23333333333333303, 'p': 0.233333333333..."
10224,"{'r': 0.158415841584158, 'p': 0.16080402010050...","{'r': 0.032934131736526005, 'p': 0.02917771883...","{'r': 0.15346534653465302, 'p': 0.155778894472..."
10263,"{'r': 0.19626168224299, 'p': 0.375, 'f': 0.257...","{'r': 0.09202453987730001, 'p': 0.174418604651...","{'r': 0.177570093457943, 'p': 0.33928571428571..."
...,...,...,...
9656,"{'r': 0.359223300970873, 'p': 0.28172588832487...","{'r': 0.09090909090909001, 'p': 0.069912609238...","{'r': 0.34304207119741104, 'p': 0.269035532994..."
9663,"{'r': 0.28534031413612504, 'p': 0.269135802469...","{'r': 0.062581486310299, 'p': 0.06045340050377...","{'r': 0.264397905759162, 'p': 0.24938271604938..."
9708,"{'r': 0.374045801526717, 'p': 0.42241379310344...","{'r': 0.136563876651982, 'p': 0.15346534653465...","{'r': 0.358778625954198, 'p': 0.40517241379310..."
9739,"{'r': 0.32731554160125503, 'p': 0.504842615012...","{'r': 0.19014626635873702, 'p': 0.261099365750...","{'r': 0.309262166405023, 'p': 0.47699757869249..."


,rouge-1,rouge-2,rouge-l
1004,"{'r': 0.230337078651685, 'p': 0.22404371584699...","{'r': 0.07228915662650601, 'p': 0.073619631901...","{'r': 0.21910112359550502, 'p': 0.213114754098..."
1005,"{'r': 0.234042553191489, 'p': 0.28205128205128...","{'r': 0.092592592592592, 'p': 0.13586956521739...","{'r': 0.20567375886524802, 'p': 0.247863247863..."
10065,"{'r': 0.21111111111111103, 'p': 0.283582089552...","{'r': 0.07042253521126701, 'p': 0.086956521739...","{'r': 0.21111111111111103, 'p': 0.283582089552..."
10224,"{'r': 0.17821782178217802, 'p': 0.173076923076...","{'r': 0.032934131736526005, 'p': 0.02956989247...","{'r': 0.173267326732673, 'p': 0.16826923076923..."
10263,"{'r': 0.19626168224299, 'p': 0.403846153846153...","{'r': 0.08588957055214701, 'p': 0.168674698795...","{'r': 0.15887850467289702, 'p': 0.326923076923..."
...,...,...,...
9656,"{'r': 0.34304207119741104, 'p': 0.289617486338...","{'r': 0.08441558441558401, 'p': 0.069986541049...","{'r': 0.326860841423948, 'p': 0.27595628415300..."
9663,"{'r': 0.272251308900523, 'p': 0.28888888888888...","{'r': 0.056062581486310006, 'p': 0.06005586592...","{'r': 0.25654450261780104, 'p': 0.272222222222..."
9708,"{'r': 0.419847328244274, 'p': 0.43307086614173...","{'r': 0.158590308370044, 'p': 0.16363636363636...","{'r': 0.389312977099236, 'p': 0.40157480314960..."
9739,"{'r': 0.328100470957613, 'p': 0.49821215733015...","{'r': 0.183987682832948, 'p': 0.25698924731182...","{'r': 0.309262166405023, 'p': 0.46960667461263..."


Wall time: 98 ms


### Google ROUGE

In [12]:
%%time
# Calculate Google ROUGE score for the whole dataset for the first pipeline (Simplify & Summarize)
calculate_metrics_for_dataset(google_rouge_scores, pipeline=p1)

CPU times: user 11min 5s, sys: 3.72 s, total: 11min 9s
Wall time: 11min 10s


In [13]:
%%time
# Calculate Google ROUGE score for the whole dataset for the second pipeline (Summarize & Simplify)
calculate_metrics_for_dataset(google_rouge_scores, pipeline=p2)

CPU times: user 10min 18s, sys: 2.73 s, total: 10min 21s
Wall time: 10min 22s


In [14]:
%%time
simplify_summary_df_google = pandas.read_json(
    'data/simplify_summary_google_rouge_scores',
    encoding='utf8',
    orient='index'
)
display(simplify_summary_df_google)

summary_simplify_df_google = pandas.read_json(
    'data/summary_simplify_google_rouge_scores',
    encoding='utf8',
    orient='index'
)
display(summary_simplify_df_google)

,rouge-1,rouge-2,rouge-l,rouge-lsum
1004,"{'p': 0.48948948948948906, 'r': 0.307547169811...","{'p': 0.12951807228915602, 'r': 0.081285444234...","{'p': 0.273273273273273, 'r': 0.17169811320754...","{'p': 0.48048048048048003, 'r': 0.301886792452..."
1005,"{'p': 0.601990049751243, 'r': 0.26419213973799...","{'p': 0.25, 'r': 0.10940919037199101, 'f': 0.1...","{'p': 0.42288557213930306, 'r': 0.185589519650...","{'p': 0.577114427860696, 'r': 0.25327510917030..."
10065,"{'p': 0.355263157894736, 'r': 0.35064935064935...","{'p': 0.07947019867549601, 'r': 0.078431372549...","{'p': 0.19736842105263103, 'r': 0.194805194805...","{'p': 0.342105263157894, 'r': 0.33766233766233..."
10224,"{'p': 0.45040214477211804, 'r': 0.457765667574...","{'p': 0.09408602150537601, 'r': 0.095628415300...","{'p': 0.193029490616621, 'r': 0.19618528610354...","{'p': 0.42091152815013405, 'r': 0.427792915531..."
10263,"{'p': 0.6666666666666661, 'r': 0.3440860215053...","{'p': 0.284210526315789, 'r': 0.14594594594594...","{'p': 0.39583333333333304, 'r': 0.204301075268...","{'p': 0.645833333333333, 'r': 0.33333333333333..."
...,...,...,...,...
9656,"{'p': 0.44308035714285704, 'r': 0.575362318840...","{'p': 0.11061452513966401, 'r': 0.143686502177...","{'p': 0.15959821428571402, 'r': 0.207246376811...","{'p': 0.4296875, 'r': 0.557971014492753, 'f': ..."
9663,"{'p': 0.48091603053435106, 'r': 0.473684210526...","{'p': 0.09061135371179001, 'r': 0.089247311827...","{'p': 0.17121046892039202, 'r': 0.168635875402...","{'p': 0.46564885496183206, 'r': 0.458646616541..."
9708,"{'p': 0.556521739130434, 'r': 0.49612403100775...","{'p': 0.24017467248908203, 'r': 0.214007782101...","{'p': 0.339130434782608, 'r': 0.30232558139534...","{'p': 0.5434782608695651, 'r': 0.4844961240310..."
9739,"{'p': 0.7728781412991941, 'r': 0.5663655316191...","{'p': 0.422201138519924, 'r': 0.30935001737921...","{'p': 0.39971550497866204, 'r': 0.292911744266...","{'p': 0.752963489805595, 'r': 0.55177206393328..."


,rouge-1,rouge-2,rouge-l,rouge-lsum
1004,"{'p': 0.5240963855421681, 'r': 0.3283018867924...","{'p': 0.151057401812688, 'r': 0.09451795841209...","{'p': 0.271084337349397, 'r': 0.16981132075471...","{'p': 0.515060240963855, 'r': 0.32264150943396..."
1005,"{'p': 0.6190476190476191, 'r': 0.2554585152838...","{'p': 0.25531914893617, 'r': 0.105032822757111...","{'p': 0.42328042328042303, 'r': 0.174672489082...","{'p': 0.592592592592592, 'r': 0.24454148471615..."
10065,"{'p': 0.392307692307692, 'r': 0.33116883116883...","{'p': 0.093023255813953, 'r': 0.07843137254901...","{'p': 0.223076923076923, 'r': 0.18831168831168...","{'p': 0.37692307692307603, 'r': 0.318181818181..."
10224,"{'p': 0.47154471544715404, 'r': 0.474114441416...","{'p': 0.09239130434782601, 'r': 0.092896174863...","{'p': 0.17886178861788601, 'r': 0.179836512261...","{'p': 0.441734417344173, 'r': 0.44414168937329..."
10263,"{'p': 0.691489361702127, 'r': 0.34946236559139...","{'p': 0.27956989247311803, 'r': 0.140540540540...","{'p': 0.446808510638297, 'r': 0.22580645161290...","{'p': 0.659574468085106, 'r': 0.33333333333333..."
...,...,...,...,...
9656,"{'p': 0.47459727385377903, 'r': 0.555072463768...","{'p': 0.11290322580645101, 'r': 0.132075471698...","{'p': 0.16976456009913202, 'r': 0.198550724637...","{'p': 0.460966542750929, 'r': 0.53913043478260..."
9663,"{'p': 0.528205128205128, 'r': 0.44253490870032...","{'p': 0.09370988446726501, 'r': 0.078494623655...","{'p': 0.18974358974358901, 'r': 0.158968850698...","{'p': 0.512820512820512, 'r': 0.42964554242749..."
9708,"{'p': 0.5840336134453781, 'r': 0.5387596899224...","{'p': 0.261603375527426, 'r': 0.24124513618677...","{'p': 0.34873949579831903, 'r': 0.321705426356...","{'p': 0.5672268907563021, 'r': 0.5232558139534..."
9739,"{'p': 0.7849037987173161, 'r': 0.5528144544822...","{'p': 0.42991115498519206, 'r': 0.302745915884...","{'p': 0.412925505673408, 'r': 0.29082696316886...","{'p': 0.7631968426245681, 'r': 0.5375260597637..."


Wall time: 58 ms


In [15]:
def calculate_avg_rouge_stats(rouge_dataframe, type="normal"):
    '''
    Calculate the average ROUGE scores r,p,f for the whole given dataset 
    '''
    avg_rouge_stats = dict(
    {
        'rouge-1': {
            "r": 0,
            "p": 0,
            "f": 0,
        },
        'rouge-2': {
            "r": 0,
            "p": 0,
            "f": 0,
        },
        'rouge-l': {
            "r": 0,
            "p": 0,
            "f": 0,
        }
    })
    if type == "google":
        avg_rouge_stats['rouge-lsum'] = {
            "r": 0,
            "p": 0,
            "f": 0,
        }

    dataframe_size = len(rouge_dataframe)
    for i in range(dataframe_size):
        for key in avg_rouge_stats:
            avg_rouge_stats[key]['r'] = avg_rouge_stats[key]['r'] + rouge_dataframe.iloc[i][key]['r']
            avg_rouge_stats[key]['p'] = avg_rouge_stats[key]['p'] + rouge_dataframe.iloc[i][key]['p']
            avg_rouge_stats[key]['f'] = avg_rouge_stats[key]['f']+ rouge_dataframe.iloc[i][key]['f']

    for key in avg_rouge_stats:
        avg_rouge_stats[key]['r'] = round(avg_rouge_stats[key]['r'] / dataframe_size, 3)
        avg_rouge_stats[key]['p'] = round(avg_rouge_stats[key]['p'] / dataframe_size, 3)
        avg_rouge_stats[key]['f'] = round(avg_rouge_stats[key]['f'] / dataframe_size, 3)
    return avg_rouge_stats

In [16]:
%%time
print("The ROUGE-1,ROUGE-2 and ROUGE-l score for the Simplify & Summary : ")
pprint(calculate_avg_rouge_stats(simplify_summary_df))

print("The ROUGE-1,ROUGE-2 and ROUGE-l score for the Summary & Simplify : ")
pprint(calculate_avg_rouge_stats(summary_simplify_df))

The ROUGE-1,ROUGE-2 and ROUGE-l score for the Simplify & Summary : 
{'rouge-1': {'f': 0.309, 'p': 0.344, 'r': 0.297},
 'rouge-2': {'f': 0.129, 'p': 0.145, 'r': 0.125},
 'rouge-l': {'f': 0.292, 'p': 0.326, 'r': 0.28}}
The ROUGE-1,ROUGE-2 and ROUGE-l score for the Summary & Simplify : 
{'rouge-1': {'f': 0.309, 'p': 0.352, 'r': 0.291},
 'rouge-2': {'f': 0.13, 'p': 0.151, 'r': 0.123},
 'rouge-l': {'f': 0.293, 'p': 0.333, 'r': 0.275}}
Wall time: 853 ms


In [17]:
%%time
print("The Google ROUGE-1, ROUGE-2, ROUGE-l and ROUGE-Lsum score for the Simplify & Summary : ")
pprint(calculate_avg_rouge_stats(simplify_summary_df_google,'google'))

print("The Google  ROUGE-1, ROUGE-2, ROUGE-l and ROUGE-Lsum score for the Summary & Simplify : ")
pprint(calculate_avg_rouge_stats(summary_simplify_df_google,'google'))

The Google ROUGE-1, ROUGE-2, ROUGE-l and ROUGE-Lsum score for the Simplify & Summary : 
{'rouge-1': {'f': 0.513, 'p': 0.583, 'r': 0.495},
 'rouge-2': {'f': 0.23, 'p': 0.267, 'r': 0.217},
 'rouge-l': {'f': 0.271, 'p': 0.317, 'r': 0.256},
 'rouge-lsum': {'f': 0.498, 'p': 0.566, 'r': 0.48}}
The Google  ROUGE-1, ROUGE-2, ROUGE-l and ROUGE-Lsum score for the Summary & Simplify : 
{'rouge-1': {'f': 0.509, 'p': 0.594, 'r': 0.478},
 'rouge-2': {'f': 0.228, 'p': 0.274, 'r': 0.21},
 'rouge-l': {'f': 0.272, 'p': 0.326, 'r': 0.251},
 'rouge-lsum': {'f': 0.494, 'p': 0.576, 'r': 0.464}}
Wall time: 1.01 s


## Semantic similarity scores

In [ ]:
!pip install jupyterlab pandas datasets matplotlib plotly scikit-learn tqdm ipywidgets 
!pip install numpy spacy textdistance fasttext 
!pip install tensorflow tensorflow_hub sentence-transformers openai
!conda install pyemd gensim

# Download the Spacy Model
!python -m spacy download en_core_web_sm

Code based on [this](https://towardsdatascience.com/semantic-textual-similarity-83b3ca4a840e) turorial:

### Word Movers Distance (WMD)

> 💡 Word embeddings are models that encode words into numeric vectors such that similar words have vectors that are near each other in vector space.

There are several ways to generate word embeddings, the most prominent being Word2Vec, GloVe, and FastText.

Since we need to compare the similarity between texts that contain multiple words, the simplest way to go from individual word embeddings into a single sentence embedding is to calculate the element-wise average of all the word embeddings in that text. However, there is an even better approach to computing the similarity between texts directly from the word embeddings called Word Movers Distance (WMD).

[WMD](http://proceedings.mlr.press/v37/kusnerb15.pdf) is based on the concept of [Earth Movers Distance](https://www.cs.jhu.edu/~misha/Papers/Rubner98.pdf) and is the minimum distance that the word embeddings from one document need to “travel” to reach the word embeddings of the document we are comparing it to. Since each document includes multiple words, the WMD calculation needs to calculate the distances from each word to every other word. It also weights the “travel” by the term frequencies of each word. Thankfully, the [gensim](https://github.com/RaRe-Technologies/gensim) library implements this complex computation efficiently using the [Fast WMD algorithm](https://www.cs.huji.ac.il/w~werman/Papers/ICCV2009.pdf). We can easily use it with just a single line of code!

Though we can use any word embedding model with WMD, I decide to use the [FastText model](https://arxiv.org/pdf/1607.04606.pdf) pre-trained on Wikipedia primarily because FastText uses sub-word information and will never run into Out Of Vocabulary issues that Word2Vec or GloVe might encounter. Take note to preprocess the texts to remove stopwords, lower case, and lemmatize them to ensure that the WMD calculation only uses informative words. Finally, since the WMD is a distance metric while we are looking for a similarity metric, we multiply the WMD value by -1 (Negative WMD) so that more similar texts have numerically larger values.

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import numpy as np
import gensim.downloader as api

nlp = spacy.load("en_core_web_sm")

In [6]:
def text_processing(sentence):
    sentence = [token.lemma_.lower()
                for token in nlp(sentence) 
                if token.is_alpha and not token.is_stop]   
    return sentence

def cos_sim(sentence1_emb, sentence2_emb):
    cos_sim = cosine_similarity(sentence1_emb, sentence2_emb)
    return np.diag(cos_sim)

!! Run the next cell only once to load the model !!

In [9]:
%%time
# Load the pre-trained model
model = api.load('fasttext-wiki-news-subwords-300')

Wall time: 5min 15s


In [7]:
def wmd_scores(file1, file2):
    # load text from file as a gle sentence
    text1 = ' '.join(read_file_as_text(file1).split('\n'))
    text2 = ' '.join(read_file_as_text(file2).split('\n'))
    # Text Processing
    processed_text1 = text_processing(text1)
    processed_text2 = text_processing(text2)
    
    # Negative Word Movers Distance
    return -model.wmdistance(processed_text1, processed_text2)

In [10]:
%%time
# Calculate WMD similarity for the whole dataset for the first pipeline (Simplify & Summarize)
wmd_p1 = calculate_metrics_for_dataset(wmd_scores, pipeline=p1)

Done generating wmd_scores scores for 99569
Done
Wall time: 1h 27min 57s


In [11]:
%%time
# Calculate WMD similarity for the whole dataset for the second pipeline (Summarize & Simplify)
wmd_p2 = calculate_metrics_for_dataset(wmd_scores, pipeline=p2)

Done generating wmd_scores scores for 99569
Done
Wall time: 1h 15min 23s


In [ ]:
%%time
with open('data/simplify_summary_wmd_scores', 'r', encoding="utf8") as p1_f:
    wmd_p1 = json.load(p1_f)

with open('data/summary_simplify_wmd_scores', 'r', encoding="utf8") as p2_f:
    wmd_p2 = json.load(p2_f)

In [12]:
def get_basic_wmd_stats(wmd_scores):
	stats = {}
	stats['average'] = round(sum(wmd_scores.values())/len(wmd_scores), 3)
	stats['max_score'] = round(max(wmd_scores.values()), 3)
	stats['min_score'] = round(min(wmd_scores.values()), 3)
	return stats

In [13]:
wmd_p1_stats = get_basic_wmd_stats(wmd_p1)
wmd_p2_stats = get_basic_wmd_stats(wmd_p2)

In [14]:
print('WMD Similarity score stats for pipeline 1: ' , wmd_p1_stats)
print('WMD Similarity score stats for pipeline 2: ' , wmd_p2_stats)

WMD Similarity score stats for pipeline 1:  {'average': -0.589, 'max_score': -0.27, 'min_score': -1.028}
WMD Similarity score stats for pipeline 2:  {'average': -0.592, 'max_score': -0.242, 'min_score': -0.993}


## Cross encoder

In [15]:
%%time
from sentence_transformers import CrossEncoder

# Load the pre-trained model
model = CrossEncoder('cross-encoder/stsb-roberta-base')

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/142 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Wall time: 26.2 s


In [ ]:
def sbert_x_encoder_scores(file1, file2):
    # load text from file as a gle sentence
    text1 = ' '.join(read_file_as_text(file1).split('\n'))
    text2 = ' '.join(read_file_as_text(file2).split('\n'))
    # Text Processing
    processed_text1 = text_processing(text1)
    processed_text2 = text_processing(text2)
    
    # Negative Word Movers Distance
    return model.predict([processed_text1, processed_text2], show_progress_bar=True)

In [ ]:
pprint(sbert_x_encoder_scores('data/wiki-auto/25/simplify_summary/simplified_summary.txt','data/wiki-auto/25/destination.txt'))